# Clustering Consulting Project 

A large technology firm needs your help, they've been hacked! Luckily their forensic engineers have grabbed valuable data about the hacks, including information like session time,locations, wpm typing speed, etc. The forensic engineer relates to you what she has been able to figure out so far, she has been able to grab meta data of each session that the hackers used to connect to their servers. These are the features of the data:

* 'Session_Connection_Time': How long the session lasted in minutes
* 'Bytes Transferred': Number of MB transferred during session
* 'Kali_Trace_Used': Indicates if the hacker was using Kali Linux
* 'Servers_Corrupted': Number of server corrupted during the attack
* 'Pages_Corrupted': Number of pages illegally accessed
* 'Location': Location attack came from (Probably useless because the hackers used VPNs)
* 'WPM_Typing_Speed': Their estimated typing speed based on session logs.


The technology firm has 3 potential hackers that perpetrated the attack. Their certain of the first two hackers but they aren't very sure if the third hacker was involved or not. They have requested your help! Can you help figure out whether or not the third suspect had anything to do with the attacks, or was it just two hackers? It's probably not possible to know for sure, but maybe what you've just learned about Clustering can help!

**One last key fact, the forensic engineer knows that the hackers trade off attacks. Meaning they should each have roughly the same amount of attacks. For example if there were 100 total attacks, then in a 2 hacker situation each should have about 50 hacks, in a three hacker situation each would have about 33 hacks. The engineer believes this is the key element to solving this, but doesn't know how to distinguish this unlabeled data into groups of hackers.**

In [1]:
from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('NumHackers').getOrCreate()

In [6]:
df = spark.read.csv('hack_data.csv', inferSchema = True, header = True)

In [8]:
print (df.columns)
df.printSchema()

['Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted', 'Location', 'WPM_Typing_Speed']
root
 |-- Session_Connection_Time: double (nullable = true)
 |-- Bytes Transferred: double (nullable = true)
 |-- Kali_Trace_Used: integer (nullable = true)
 |-- Servers_Corrupted: double (nullable = true)
 |-- Pages_Corrupted: double (nullable = true)
 |-- Location: string (nullable = true)
 |-- WPM_Typing_Speed: double (nullable = true)



In [9]:
df.describe().show()

+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|summary|Session_Connection_Time| Bytes Transferred|   Kali_Trace_Used|Servers_Corrupted|   Pages_Corrupted|   Location|  WPM_Typing_Speed|
+-------+-----------------------+------------------+------------------+-----------------+------------------+-----------+------------------+
|  count|                    334|               334|               334|              334|               334|        334|               334|
|   mean|     30.008982035928145| 607.2452694610777|0.5119760479041916|5.258502994011977|10.838323353293413|       null|57.342395209580864|
| stddev|     14.088200614636158|286.33593163576757|0.5006065264451406| 2.30190693339697|  3.06352633036022|       null| 13.41106336843464|
|    min|                    1.0|              10.0|                 0|              1.0|               6.0|Afghanistan|              40.0|
|    max|           

In [10]:
# Features = 'Session_Connection_Time', 'Bytes Transferred', 'Kali_Trace_Used', 'Servers_Corrupted', 'Pages_Corrupted'
                # 'Location', 'WPM_Typing_Speed'

In [12]:
df.select('Location').distinct().count()
# since location vs total set is small, we will remove this feature

181

In [13]:
# Data Standardizer: 'Session_Connection_Time', 'Bytes Transferred', 'Servers_Corrupted', 'WPM_Typing_Speed', 'Pages_Corrupted'
# None: Kali_Trace_Used

In [26]:
# Libraries
from pyspark.ml.linalg import Vectors
from pyspark.ml.feature import VectorAssembler
from pyspark.ml.feature import StandardScaler
from pyspark.ml.clustering import KMeans
from pyspark.ml.evaluation import ClusteringEvaluator
from pyspark.ml import Pipeline

In [56]:
# Pipeline Stages - 1

# Assembler - 1
assembler_1 = VectorAssembler(inputCols = ['Session_Connection_Time', 'Bytes Transferred', 
                                           'Servers_Corrupted', 'WPM_Typing_Speed', 'Pages_Corrupted']
                              , outputCol = 'featuresToScale')

# Data Sndardizer
scaler = StandardScaler(inputCol="featuresToScale", outputCol="scaledFeatures",
                        withStd=True, withMean=False)

# Assembler - 2
assembler_2 = VectorAssembler(inputCols = ['Kali_Trace_Used', 'scaledFeatures']
                              , outputCol = 'features')

In [59]:
# Pipeline Stages - 2

# Machine Learning Model
kmeans_1 = KMeans(k=2, seed=1)
kmeans_2 = KMeans(k=3, seed=1)

In [60]:
#pipeline

pipeline_1 = Pipeline(stages=[assembler_1, scaler, assembler_2, kmeans_1])
pipeline_2 = Pipeline(stages=[assembler_1, scaler, assembler_2, kmeans_2])

In [61]:
model_1 = pipeline_1.fit(df)
model_2 = pipeline_2.fit(df)

In [62]:
kmeans_1_model = model_1.stages[-1].summary.predictions
kmeans_2_model = model_2.stages[-1].summary.predictions

In [63]:
# Analysis
kmeans_1_model.groupBy('prediction').count().show()
kmeans_2_model.groupBy('prediction').count().show()

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  167|
|         0|  167|
+----------+-----+

+----------+-----+
|prediction|count|
+----------+-----+
|         1|  102|
|         2|   65|
|         0|  167|
+----------+-----+



In [64]:
print ("Number of hackers are 2")

Number of hackers are 2


In [65]:
# i can try one mor thing, look at the cost, and see the cost as k increases, logically k inc, cost dec, 
# upto a right number of clusters; then relationship might change